In [ ]:
# %pip install datasets

In [ ]:
from datasets import load_dataset

In [ ]:
import model_one

In [ ]:
model_one.API_KEY = 'eyJhb...'

In [ ]:
dataset = load_dataset("tweet_eval", "irony")
model = model_one.create_classifier(r'tweet_eval-irony.json')
model

In [ ]:
model.upload_arrays(dataset['train']['text'], dataset['train']['label'],
                    dataset['test']['text'], dataset['test']['label'])
model

In [ ]:
model.fit()

In [ ]:
model.classify(dataset['validation']['text'][0])